## Baseball Prediction: 1 - Data Wrangling
In this notebook we will wrangle data downloaded from www.retrosheet.org into a dataframe suitable for model building.  Specifically, for each game, we will calculate some team statistics over their past 162 games.

At the end, we save our dataframe to a file.  This file will be the starting point for the next notebook, in which we build our first model.

To use this notebook, you must first download the game logs here: https://www.retrosheet.org/gamelogs/index.html
Towards the bottom of the page there are links for ZIP files containing multiple seasons.  Download the 5 zip files for 1980-1989, 1990-1999, ... , 2020-2022, decompress them, and then move all of the single season files to a single directory.  You will then need the path to that directory for the variable `fname` below.

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [14]:
# Fill in the path to your data here...
fname = '~/Desktop/MLB-YT/data/'+'gl2022.txt'
df = pd.read_csv(fname, header=None)

In [15]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,20220407,0,Thu,SDN,NL,1,ARI,NL,1,2,4,51,N,NaN,NaN,NaN,PHO01,35508.0,198,001100000,000000004,29,4,1,0,0,2,0,0,0,7,0,5,1,0,2,0,7,5,4,4,2,0,24,8,0,0,1,0,26,3,0,0,1,3,0,0,1,7,0,6,0,0,1,0,6,6,2,2,0,0,27,9,0,0,2,0,vanol901,Larry Vanover,belld901,Dan Bellino,barbs901,Sean Barber,valej901,Junior Valentine,NaN,(none),NaN,(none),melvb001,Bob Melvin,lovut001,Tony Lovullo,mantj002,Joe Mantiply,suarr002,Robert Suarez,NaN,(none),beers001,Seth Beer,darvy001,Yu Darvish,bumgm001,Madison Bumgarner,nolaa002,Austin Nola,2,machm001,Manny Machado,5,cronj001,Jake Cronenworth,4,voitl001,Luke Voit,10,myerw001,Wil Myers,9,hosme001,Eric Hosmer,3,profj001,Jurickson Profar,7,kim-h002,Ha-Seong Kim,6,grist001,Trent Grisham,8,varsd001,Daulton Varsho,8,martk001,Ketel Marte,4,perad001,David Peralta,7,walkc002,Christian Walker,3,smitp002,Pavin Smith,9,kellc002,Carson Kelly,2,beers001,Seth Beer,10,ellid002,Drew Ellis,5,perdg001,Gerardo Perdomo,6,NaN,Y
1,20220407,0,Thu,CIN,NL,1,ATL,NL,1,6,3,54,N,NaN,NaN,NaN,ATL03,40545.0,181,012003000,001000020,35,10,0,0,1,6,0,1,1,1,0,13,0,0,1,0,5,5,2,2,0,0,27,7,1,0,0,0,31,4,0,0,1,2,0,0,0,5,0,13,0,0,0,0,6,4,6,6,0,0,27,7,0,0,1,0,laynj901,Jerry Layne,wendh902,Hunter Wendelstedt,whitc901,Chad Whitson,hamaa901,Adam Hamari,NaN,(none),NaN,(none),belld002,David Bell,snitb801,Brian Snitker,mahlt001,Tyler Mahle,friem001,Max Fried,santt001,Tony Santillan,farmk001,Kyle Farmer,mahlt001,Tyler Mahle,friem001,Max Fried,indij001,Jonathan India,4,aquia001,Aristides Aquino,9,phamt001,Tommy Pham,7,vottj001,Joey Votto,3,stept001,Tyler Stephenson,2,senzn001,Nick Senzel,8,mousm001,Mike Moustakas,10,farmk001,Kyle Farmer,6,drurb001,Brandon Drury,5,rosae001,Eddie Rosario,9,olsom001,Matt Olson,3,rilea001,Austin Riley,5,ozunm001,Marcell Ozuna,7,albio001,Ozzie Albies,4,duvaa001,Adam Duvall,8,darnt001,Travis d'Arnaud,2,dicka001,Alex Dickerson,10,swand001,Dansby Swanson,6,NaN,Y
2,20220407,0,Thu,MIL,NL,1,CHN,NL,1,4,5,51,D,NaN,NaN,NaN,CHI11,35112.0,198,000100210,00003020x,33,10,4,0,0,4,0,2,1,4,0,9,1,0,2,0,9,3,5,5,0,0,24,10,0,0,1,0,29,8,3,0,1,5,0,1,1,4,0,7,0,1,1,0,6,6,4,4,0,0,27,9,0,0,2,0,barrt901,Ted Barrett,barkl901,Lance Barksdale,lentn901,Nic Lentz,cejan901,Nestor Ceja,NaN,(none),NaN,(none),counc001,Craig Counsell,rossd001,David Ross,givem001,Mychal Givens,ashba003,Aaron Ashby,robed002,David Robertson,happi001,Ian Happ,burnc002,Corbin Burnes,hendk001,Kyle Hendricks,wongk001,Kolten Wong,4,adamw002,Willy Adames,6,yelic001,Christian Yelich,7,mccua001,Andrew McCutchen,10,tellr001,Rowdy Tellez,3,renfh001,Hunter Renfroe,9,narvo001,Omar Narvaez,2,cainl001,Lorenzo Cain,8,petej002,Jace Peterson,5,orter001,Rafael Ortega,10,madrn001,Nick Madrigal,4,contw001,Willson Contreras,2,happi001,Ian Happ,7,schwf001,Frank Schwindel,3,suzus001,Seiya Suzuki,9,heywj001,Jason Heyward,8,wisdp001,Patrick Wisdom,5,hoern001,Nico Hoerner,6,NaN,Y
3,20220407,0,Thu,PIT,NL,1,SLN,NL,1,0,9,51,D,NaN,NaN,NaN,STL10,46256.0,188,000000000,13000104x,30,6,0,0,0,0,0,0,0,2,0,9,0,1,2,0,5,6,9,9,0,0,24,7,2,0,1,0,31,8,2,0,3,9,0,2,1,7,0,5,1,0,1,0,8,4,0,0,1,0,27,11,0,0,3,0,reynj901,Jim Reynolds,ticht901,Todd Tichenor,muchm901,Mike Muchlinski,merzd901,Dan Merzel,NaN,(none),NaN,(none),sheld801,Derek Shelton,marmo801,Oliver Marmol,waina001,Adam Wainwright,brubj001,JT Brubaker,NaN,(none),oneit001,Tyler O'Neill,brubj001,JT Brubaker,waina001,Adam Wainwright,voged001,Daniel Vogelbach,10,reynb001,Bryan Reynolds,8,hayek001,Ke'Bryan Hayes,5,tsuty001,Yoshi Tsutsugo,3,newmk001,Kevin Newman,6,tu

In [16]:
# Get colnames

In [17]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]


In [18]:
df.columns = colnames

In [19]:
df.sample(10)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
1670,20220811,0,Thu,MIA,NL,111,PHI,NL,111,3,0,54,D,NaN,NaN,NaN,PHI13,25444.0,187,010002000,000000000,34,7,2,1,0,2,0,0,0,2,0,8,1,0,1,0,6,4,0,0,0,0,27,8,0,0,0,0,33,7,1,0,0,0,0,0,0,4,0,13,0,1,0,0,10,4,2,2,1,0,27,5,1,0,1,0,ortir901,Roberto Ortiz,eddid901,Doug Eddings,knigb901,Brian Knight,willr901,Ryan Wills,NaN,(none),NaN,(none),mattd001,Don Mattingly,thomr801,Rob Thomson,cabre003,Edward Cabrera,gibsk002,Kyle Gibson,scott003,Tanner Scott,wendj002,Joey Wendle,cabre003,Edward Cabrera,gibsk002,Kyle Gibson,wendj002,Joey Wendle,4,rojam002,Miguel Rojas,6,coopg002,Garrett Cooper,10,bledj001,JJ Bleday,9,fortn001,Nick Fortes,2,leblc001,Charles Leblanc,5,diazl001,Lewin Diaz,3,burdp001,Peyton Burdick,8,willl001,Luke Williams,7,schwk001,Kyle Schwarber,7,hoskr001,Rhys Hoskins,3,bohma001,Alec Bohm,5,halld003,Darick Hall,10,castn001,Nick Castellanos,9,stotb001,Bryson Stott,6,seguj002,Jean Segura,4,stubg001,Garrett Stubbs,2,vierm001,Matt Vierling,8,NaN,Y
928,20220615,0,Wed,CIN,NL,63,ARI,NL,65,4,7,51,D,NaN,NaN,NaN,PHO01,14917.0,170,020000011,03000004x,31,5,1,0,1,3,0,0,4,0,0,6,0,0,0,0,4,3,7,7,0,1,24,9,1,0,0,0,33,8,3,0,1,6,1,0,0,2,0,6,2,0,0,0,5,3,4,4,1,0,27,7,2,0,0,0,mahrn901,Nick Mahrley,carav901,Vic Carapazza,mealj901,Jerry Meals,viscj901,Jansen Visconti,NaN,(none),NaN,(none),belld002,David Bell,lovut001,Tony Lovullo,ramin002,Noe Ramirez,warra002,Art Warren,NaN,(none),perad001,David Peralta,castl003,Luis Castillo,gallz001,Zac Gallen,indij001,Jonathan India,4,drurb001,Brandon Drury,5,reynm003,Matt Reynolds,6,vottj001,Joey Votto,10,mousm001,Mike Moustakas,3,senzn001,Nick Senzel,8,almoa002,Albert Almora,7,friet001,TJ Friedl,9,garca006,Aramis Garcia,2,varsd001,Daulton Varsho,9,rojaj001,Josh Rojas,5,martk001,Ketel Marte,4,walkc002,Christian Walker,3,perad001,David Peralta,7,smitp002,Pavin Smith,10,thoma004,Alek Thomas,8,perdg001,Gerardo Perdomo,6,herrj003,Jose Herrera,2,NaN,Y
789,20220604,0,Sat,HOU,AL,53,KCA,AL,51,0,6,51,D,NaN,NaN,NaN,KAN06,14686.0,190,000000000,00000204x,32,5,1,0,0,0,0,0,0,5,0,5,1,0,1,0,10,3,6,6,1,0,24,6,0,0,0,0,34,10,2,0,1,6,0,0,0,1,0,4,1,0,0,0,5,5,0,0,0,0,27,11,1,0,1,0,hicke901,Ed Hickox,carav901,Vic Carapazza,willr901,Ryan Wills,mealj901,Jerry Meals,NaN,(none),NaN,(none),baked002,Dusty Baker,mathm001,Mike Matheny,snidc001,Collin Snider,garcl007,Luis Garcia,NaN,(none),peres002,Salvador Per

## Create a Table with every game since 1980

In [20]:
df = pd.DataFrame()
for year in range(1980,2023):
    fname = '~/Desktop/MLB-YT/data/gl' +str(year)+'.txt'
    df_temp = pd.read_csv(fname, header=None)
    df_temp.columns = colnames
    df_temp['season'] = year
    df = pd.concat((df, df_temp))


In [21]:
df.shape

(96276, 162)

In [22]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
Index: 96276 entries, 0 to 2429
Data columns (total 162 columns):
 #    Column                Non-Null Count  Dtype  
---   ------                --------------  -----  
 0    date                  96276 non-null  int64  
 1    dblheader_code        96276 non-null  int64  
 2    day_of_week           96276 non-null  object 
 3    team_v                96276 non-null  object 
 4    league_v              96276 non-null  object 
 5    game_no_v             96276 non-null  int64  
 6    team_h                96276 non-null  object 
 7    league_h              96276 non-null  object 
 8    game_no_h             96276 non-null  int64  
 9    runs_v                96276 non-null  int64  
 10   runs_h                96276 non-null  int64  
 11   outs_total            96276 non-null  int64  
 12   day_night             96276 non-null  object 
 13   completion_info       83 non-null     object 
 14   forfeit_info          1 non-null      object 
 15   protes

In [23]:
# Create columns for outcomes

In [24]:
## Calculate a few useful columns
df['run_diff'] = df['runs_h']-df['runs_v']
df['home_victory'] = (df['run_diff']>0).astype(int)
df['run_total'] = df['runs_h'].copy()+df['runs_v'].copy()
df['date_dblhead'] = (df['date'].astype(str) + df['dblheader_code'].astype(str)).astype(int)


In [25]:
# Do some basic exploration

In [26]:
df.home_victory.mean()

0.538368856205077

### Big picture - have outcome, need features
- Would like to start simple, base features on team results over past *n* games
- Need to do a bit of "data wrangling"
- First, we need to have each team's games in chronological order
- Then can use pandas 'rolling' functionality to aggregate over windows in the past
- Immediate goal, get batting average, on-base percentage, and slugging percentage
- Need "running values" based on the past, not including the current game

In [27]:
# Let's look at a single team's games

In [28]:
df_mets = df.loc[((df.team_v=='NYN') | (df.team_h=='NYN')), :]

In [29]:
df_mets.shape

(6736, 166)

In [30]:
df_mets.head(100)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead
8,19800410,0,Thu,CHN,NL,1,NYN,NL,1,2,5,51,D,NaN,NaN,NaN,NYC17,12219.0,143,000001100,01000400x,34,7,3,0,0,2,0,0,0,1,0,3,0,0,1,0,6,3,4,4,0,0,24,12,1,0,1,0,29,8,1,0,0,4,0,1,0,6,2,5,0,2,1,0,7,2,2,2,0,0,27,9,1,0,1,0,kiblj901,John Kibler,froeb901,Bruce Froemming,tatat901,Terry Tata,rennd901,Dutch Rennert,NaN,(none),NaN,(none),gomep101,Preston Gomez,torrj101,Joe Torre,swanc001,Craig Swan,reusr001,Rick Reuschel,allen001,Neil Allen,NaN,(none),reusr001,Rick Reuschel,swanc001,Craig Swan,randl101,Lenny Randle,4,dejei001,Ivan De Jesus,6,buckb001,Bill Buckner,3,kingd001,Dave Kingman,7,hendk101,Ken Henderson,9,ontis101,Steve Ontiveros,5,lezcc101,Carlos Lezcano,8,blact101,Tim Blackwell,2,reusr001,Rick Reuschel,1,tavef101,Frank Taveras,6,madde101,Elliott Maddox,5,mazzl001,Lee Mazzilli,3,hends001,Steve Henderson,7,jorgm001,Mike Jorgensen,9,steaj001,John Stearns,2,moraj101,Jerry Morales,8,flynd001,Doug Flynn,4,swanc001,Craig Swan,1,NaN,Y,1980,3,1,7,198004100
19,19800411,0,Fri,CHN,NL,2,NYN,NL,2,7,5,54,D,NaN,NaN,NaN,NYC17,4460.0,168,023100100,012000101,37,11,1,0,5,7,0,0,0,7,1,5,1,0,1,0,10,3,5,5,1,0,27,17,0,0,1,0,38,13,3,0,0,5,1,0,0,3,0,8,1,0,1,0,10,5,7,7,0,0,27,12,0,0,1,0,froeb901,Bruce Froemming,tatat901,Terry Tata,rennd901,Dutch Rennert,brocf901,Fred Brocklander,NaN,(none),NaN,(none),gomep101,Preston Gomez,torrj101,Joe Torre,lampd001,Dennis Lamp,burrr001,Ray Burris,suttb001,Bruce Sutter,kingd001,Dave Kingman,lampd001,Dennis Lamp,burrr001,Ray Burris,randl101,Lenny Randle,4,dejei001,Ivan De Jesus,6,buckb001,Bill Buckner,3,kingd001,Dave Kingman,7,hendk101,Ken Henderson,9,martj001,Jerry Martin,8,ontis101,Steve Ontiveros,5,footb101,Barry Foote,2,lampd001,Dennis Lamp,1,tavef101,Frank Taveras,6,madde101,Elliott Maddox,5,mazzl001,Lee Mazzilli,3,hends001,Steve Henderson,7,jorgm001,Mike Jorgensen,9,steaj001,John Stearns,2,moraj101,Jerry Morales,8,flynd001,Doug Flynn,4,burrr001,Ray Burris,1,NaN,Y,1980,-2,0,12,198004110
31,19800412,0,Sat,CHN,NL,3,NYN,NL,3,6,3,54,D,NaN,NaN,NaN,NYC17,10781.0,149,000000420,010001100,37,14,1,0,2,6,0,1,0,3,1,2,1,1,3,0,8,2,2,2,0,0,27,11,2,0,1,0,29,6,0,0,2,3,0,1,1,5,1,6,1,1,0,0,6,4,5,5,2,0,27,16,2,0,4,0,tatat901,Terry Tata,rennd901,Dutch Rennert,brocf901,Fred Brocklander,kiblj901,John Kibler,NaN,(none),NaN,(none),gomep101,Preston Gomez,t

In [31]:
# Write a function to create a team-specific data frame, given the team

In [32]:
def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

visit_cols = [col for col in df.columns if not col.endswith('_h')]
visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]
home_cols = [col for col in df.columns if not col.endswith('_v')]
home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]

## This subsets the game level df by team, to aggregate team statistics easily
## We also create rolling sums with an offset, so that the rollsum number represents
## statistics up to, but not including, the game in question

def create_team_df(team):
    df_team_v = df[(df.team_v==team)]
    opponent = df_team_v['team_h']
    df_team_v = df_team_v[visit_cols]
    df_team_v.columns = visit_cols_stripped
    df_team_v['home_game'] = 0
    df_team_v['opponent'] = opponent

    df_team_h = df[(df.team_h==team)]
    opponent = df_team_h['team_v']
    df_team_h = df_team_h[home_cols]
    df_team_h.columns = home_cols_stripped
    df_team_h['home_game'] = 1
    df_team_h['opponent'] = opponent


    df_team = pd.concat((df_team_h, df_team_v))
    df_team.sort_values(['date', 'game_no'],inplace=True)
    
    for winsize in [162,30]:
        suff = str(winsize)
        for raw_col in ['AB','H','2B','3B','HR','BB','runs','SB','CS','ERR']:
            new_col = 'rollsum_'+raw_col+'_'+suff
            df_team[new_col] = df_team[raw_col].rolling(winsize, closed='left').sum()

        df_team['rollsum_BATAVG_'+suff] = df_team['rollsum_H_'+suff] / df_team['rollsum_AB_'+suff]
        df_team['rollsum_OBP_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_BB_'+suff]) / (
                                    df_team['rollsum_AB_'+suff]+df_team['rollsum_BB_'+suff])
        df_team['rollsum_SLG_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_2B_'+suff] 
                                 + 2*df_team['rollsum_3B_'+suff]+ 
                                3*df_team['rollsum_HR_'+suff] ) / (df_team['rollsum_AB_'+suff])
        df_team['rollsum_OBS_'+suff] = df_team['rollsum_OBP_'+suff] + df_team['rollsum_SLG_'+suff]
    
    df_team['season_game'] = df_team['season']*1000 + df_team['game_no']
    df_team.set_index('season_game', inplace=True)
    return(df_team)

In [33]:
df_mets = create_team_df('NYN')

In [34]:
df_mets.sample(10)

,date,dblheader_code,day_of_week,team,league,game_no,runs,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore,AB,H,2B,3B,HR,RBI,SH,SF,HBP,BB,IBB,SO,SB,CS,GIDP,CI,LOB,P_num,ERind,ERteam,WP,balk,PO,ASST,ERR,PB,DP,TP,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id,mgr_name,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id,pitcher_start_name,batter1_name,batter1_id,batter1_pos,batter2_name,batter2_id,batter2_pos,batter3_name,batter3_id,batter3_pos,batter4_name,batter4_id,batter4_pos,batter5_name,batter5_id,batter5_pos,batter6_name,batter6_id,batter6_pos,batter7_name,batter7_id,batter7_pos,batter8_name,batter8_id,batter8_pos,batter9_name,batter9_id,batter9_pos,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,home_game,opponent,rollsum_AB_162,rollsum_H_162,rollsum_2B_162,rollsum_3B_162,rollsum_HR_162,rollsum_BB_162,rollsum_runs_162,rollsum_SB_162,rollsum_CS_162,rollsum_ERR_162,rollsum_BATAVG_162,rollsum_OBP_162,rollsum_SLG_162,rollsum_OBS_162,rollsum_AB_30,rollsum_H_30,rollsum_2B_30,rollsum_3B_30,rollsum_HR_30,rollsum_BB_30,rollsum_runs_30,rollsum_SB_30,rollsum_CS_30,rollsum_ERR_30,rollsum_BATAVG_30,rollsum_OBP_30,rollsum_SLG_30,rollsum_OBS_30
season_game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1993133,19930901,0,Wed,NYN,NL,133,2,54,D,NaN,NaN,NaN,NYC17,15851.0,161,100000010,32,8,2,0,1,2,1,0,0,3,0,4,0,1,1,0,7,2,3,3,0,0,27,6,0,0,2,0,poncl901,Larry Poncino,riekr901,Rich Rieker,davib902,Bob Davidson,hohnb901,Bill Hohn,NaN,(none),NaN,(none),greed101,Dallas Green,drabd001,Doug Drabek,ferns001,Sid Fernandez,joned001,Doug Jones,candc001,Casey Candaele,ferns001,Sid Fernandez,orsuj001,Joe Orsulak,7,hundt001,Todd Hundley,2,murre001,Eddie Murray,3,bonib001,Bobby Bonilla,5,burnj001,Jeromy Burnitz,9,kentj001,Jeff Kent,4,mcknj001,Jeff McKnight,6,thomr004,Ryan Thompson,8,ferns001,Sid Fernandez,1,NaN,Y,1993,-1,0,5,199309010,1,HOU,5383.0,1314.0,235.0,39.0,141.0,458.0,664.0,96.0,52.0,143.0,0.244102,0.303373,0.380829,0.684201,1007.0,245.0,45.0,12.0,34.0,81.0,131.0,5.0,7.0,34.0,0.243297,0.299632,0.413108,0.712741
1983112,19830809,0,Tue,NYN,NL,112,3,51,N,NaN,NaN,NaN,MON02,36870.0,169,002001000,34,11,3,0,0,3,1,1,0,2,0,7,0,1,0,0,8,2,5,5,0,0,24,14,2,1,3,0,tatat901,Terry Tata,willc901,Charlie Williams,kiblj901,John Kibler,froeb901,Bruce Froemming,NaN,(none),NaN,(none),howaf102,Frank Howard,lea-c001,Charlie Lea,lynce001,Ed Lynch,NaN,(none),cromw101,Warren Cromartie,lynce001,Ed Lynch,wilsm001,Mookie Wilson,8,brooh001,Hubie Brooks,5,hernk001,Keith Hernandez,3,fostg001,George Foster,7,strad001,Darryl Strawberry,9,bailb001,Bob Bailor,6,gileb001,Brian Giles,4,hodgr001,Ron Hodges,2,lynce001,Ed Lynch,1,NaN,Y,1983,4,1,10,198308090,0,MON,5477.0,1308.0,184.0,26.0,102.0,442.0,561.0,139.0,62.0,150.0,0.238817,0.295658,0.337776,0.633434,1008.0,242.0,21.0,4.0,22.0,80.0,104.0,23.0,12.0,24.0,0.240079,0.295956,0.334325,0.630281
1987150,19870921,0,Mon,NYN,NL,150,7,54,D,NaN,NaN,NaN,CHI11,8229.0,168,100000042,39,12,3,0,2,7,0,0,0,2,1,3,2,0,0,0,7,1,1,1,0,0,27,7,0,0,0,0,pullf901,Frank Pulli,ripps901,Steve Rippley,demud901,Dana DeMuth,bonig901,Greg Bonin,NaN,(none),NaN,(none),johnd105,Davey Johnson,goodd001,Dwight Gooden,lancl001,Les Lancaster,NaN,(none),johnh001,Howard Johnson,goodd001,Dwight Gooden,dyksl001,Lenny Dykstra,8,teuft001,Tim Teufel,4,hernk001,Keith Hernandez,3,strad001,Darryl Strawberry,9,mcrek001,Kevin McReynolds,7,cartg001,Gary Carter,2,johnh001,Howard Johnson,5,santr001,Rafael Santana,6,goodd001,Dwight Gooden,1,NaN,Y,1987,-6,0,8,198709210,0,CHN,5601.0,1501.0,277.0,35.0,196.0,605.0,825.0,147.0,47.0,141.0,0.267988,0.339349,0.434922,0.774271,1061.0,272.0,42.0,6.0,36.0,123.0,151.0,26.0,12.0,25.0,0.256362,0.333615,0.409048,0.742663
2019055,2019

In [35]:
# Make a dictionary that maps a team name to it's data frame

In [36]:
# Create the team level dataframe for each team - put in dict for easy access
team_data_dict = {}
for team in df.team_v.unique():
    team_data_dict[team] = create_team_df(team)

In [37]:
# Go through the rows of the main dataframe, and augment it with home and visiting teams' features

In [38]:
## Create a variety of summarized statistics for each game
## For each game, we look up the home and visiting team in the team
## data dictionary, and then look up the game, and pull the relevant stats

BATAVG_162_h = np.zeros(df.shape[0])
BATAVG_162_v = np.zeros(df.shape[0])
OBP_162_h = np.zeros(df.shape[0])
OBP_162_v = np.zeros(df.shape[0])
SLG_162_h = np.zeros(df.shape[0])
SLG_162_v = np.zeros(df.shape[0])
OBS_162_h = np.zeros(df.shape[0])
OBS_162_v = np.zeros(df.shape[0])
SB_162_h = np.zeros(df.shape[0])
SB_162_v = np.zeros(df.shape[0])
CS_162_h = np.zeros(df.shape[0])
CS_162_v = np.zeros(df.shape[0])
ERR_162_h = np.zeros(df.shape[0])
ERR_162_v = np.zeros(df.shape[0])
BATAVG_30_h = np.zeros(df.shape[0])
BATAVG_30_v = np.zeros(df.shape[0])
OBP_30_h = np.zeros(df.shape[0])
OBP_30_v = np.zeros(df.shape[0])
SLG_30_h = np.zeros(df.shape[0])
SLG_30_v = np.zeros(df.shape[0])
OBS_30_h = np.zeros(df.shape[0])
OBS_30_v = np.zeros(df.shape[0])
SB_30_h = np.zeros(df.shape[0])
SB_30_v = np.zeros(df.shape[0])
CS_30_h = np.zeros(df.shape[0])
CS_30_v = np.zeros(df.shape[0])
ERR_30_h = np.zeros(df.shape[0])
ERR_30_v = np.zeros(df.shape[0])
i=0
for index, row in df.iterrows():
    if i%1000==0:
        print(i)
    home_team = row['team_h']
    visit_team = row['team_v']
    game_index_v = row['season']*1000 + row['game_no_v']
    game_index_h = row['season']*1000 + row['game_no_h']
    BATAVG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_162']
    BATAVG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_162']
    OBP_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_162']
    OBP_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_162']
    SLG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_162']
    SLG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_162']
    OBS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_162']
    OBS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_162']
    SB_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_162']
    SB_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_162']
    CS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_162']
    CS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_162']
    ERR_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_162']
    ERR_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_162']
    BATAVG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_30']
    BATAVG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_30']
    OBP_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_30']
    OBP_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_30']
    SLG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_30']
    SLG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_30']
    OBS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_30']
    OBS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_30']
    SB_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_30']
    SB_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_30']
    CS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_30']
    CS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_30']
    ERR_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_30']
    ERR_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_30']
    i+=1
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000


In [39]:
## We then put the constructed arrays into the main game level dataframe
df['BATAVG_162_h'] = BATAVG_162_h
df['BATAVG_162_v'] = BATAVG_162_v
df['OBP_162_h'] = OBP_162_h
df['OBP_162_v'] = OBP_162_v
df['SLG_162_h'] = SLG_162_h
df['SLG_162_v'] = SLG_162_v
df['OBS_162_h'] = OBS_162_h
df['OBS_162_v'] = OBS_162_v
df['SB_162_h'] = SB_162_h
df['SB_162_v'] = SB_162_v
df['CS_162_h'] = CS_162_h
df['CS_162_v'] = CS_162_v
df['ERR_162_h'] = ERR_162_h
df['ERR_162_v'] = ERR_162_v
df['BATAVG_30_h'] = BATAVG_30_h
df['BATAVG_30_v'] = BATAVG_30_v
df['OBP_30_h'] = OBP_30_h
df['OBP_30_v'] = OBP_30_v
df['SLG_30_h'] = SLG_30_h
df['SLG_30_v'] = SLG_30_v
df['OBS_30_h'] = OBS_30_h
df['OBS_30_v'] = OBS_30_v
df['SB_30_h'] = SB_30_h
df['SB_30_v'] = SB_30_v
df['CS_30_h'] = CS_30_h
df['CS_30_v'] = CS_30_v
df['ERR_30_h'] = ERR_30_h
df['ERR_30_v'] = ERR_30_v


In [40]:
df.shape

(96276, 194)

In [41]:
df.sample(5)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v
1704,19830830,0,Tue,SFN,NL,132,MON,NL,130,13,2,54,N,NaN,NaN,NaN,MON02,29536.0,158,200130304,100010000,38,13,4,0,3,12,1,0,1,9,0,9,0,1,0,0,9,2,0,0,1,0,27,20,3,0,3,0,31,5,0,0,0,0,0,0,1,5,0,7,0,1,3,0,8,4,12,12,0,1,27,10,1,1,0,0,harvd901,Doug Harvey,crawj901,Jerry Crawford,grege901,Eric Gregg,pullf901,Frank Pulli,NaN,(none),NaN,(none),robif103,Frank Robinson,virdb101,Bill Virdon,davim001,Mark Davis,burrr001,Ray Burris,mintg001,Greg Minton,omalt001,Tom O'Malley,davim001,Mark Davis,burrr001,Ray Burris,venam001,Max Venable,7,younj001,Joel Youngblood,4,clarj001,Jack Clark,9,evand001,Darrell Evans,6,davic001,Chili Davis,8,omalt001,Tom O'Malley,5,bergd001,Dave Bergman,3,brenb001,Bob Brenly,2,davim001,Mark Davis,1,raint001,Tim Raines,7,trilm001,Manny Trillo,4,dawsa001,Andre Dawson,8,oliva001,Al Oliver,3,wallt001,Tim Wallach,5,wohlj001,Jim Wohlford,9,ramor001,Bobby Ramos,2,flynd001,Doug Flynn,6,burrr001,Ray Burris,1,NaN,Y,1983,-11,0,15,198308300,0.262567,0.248106,0.322692,0.325712,0.388592,0.378164,0.711283,0.703876,139.0,145.0,46.0,65.0,102.0,171.0,0.274354,0.262997,0.333333,0.328691,0.397614,0.379205,0.730948,0.707896,27.0,20.0,6.0,15.0,19.0,29.0
511,19980509,0,Sat,COL,NL,36,MON,NL,34,0,4,51,N,NaN,NaN,NaN,MON02,12014.0,140,000000000,00011101x,33,7,4,0,0,0,0,0,0,1,0,5,0,0,1,0,7,3,4,4,1,1,24,9,2,1,1,0,31,8,1,0,1,4,0,0,0,3,0,4,1,0,1,0,6,3,0,0,0,0,27,13,0,0,1,0,darlg901,Gary Darling,wintm901,Mike Winters,ripps901,Steve Rippley,poncl901,Larry Poncino,NaN,(none),NaN,(none),bayld001,Don Baylor,alouf101,Felipe Alou,moort001,Trey Moore,thomj005,John Thomson,urbiu001,Ugueth Urbina,widgc001,Chris Widger,thomj005,John Thomson,moort001,Trey Moore,lansm001,Mike Lansing,4,goodc001,Curtis Goodwin,8,burke001,Ellis Burks,7,bichd001,Dante Bichette,9,castv001,Vinny Castilla,5,colbg001,Greg Colbrunn,3,manwk001,Kirt Manwaring,2,peren001,Neifi Perez,6,thomj005,John Thomson,1,santf001,F.P. Santangelo,4,grudm001,Mark Grudzielanek,6,guerv001,Vladimir Guerrero,9,whitr001,Rondell White,8,fullb001,Brad Fullm

In [42]:
df.to_csv('df_bp1.csv', index=False)
